In [47]:
import open3d as o3d
import numpy as np
import copy
import random

# ─── 파일 경로 및 기본 설정 ─────────────────────────────
seq = "08"  # 시퀀스 고정
data_dir = "/home/workspace/KITTI/dataset/sequences/" + seq
velo_dir = data_dir + "/velodyne"
label_dir = data_dir + "/labels"
pose_path = data_dir + "/poses.txt"

# 현재 프레임 id를 30으로 설정 (파일명은 6자리 zero padding)
current_frame_id = 50

# ─── 포즈 파일 읽기 ─────────────────────────────
def load_poses(pose_file):
    poses = []
    with open(pose_file, 'r') as f:
        for line in f:
            vals = np.array(list(map(float, line.split())))
            pose = np.vstack((vals.reshape(3, 4), [0, 0, 0, 1]))  # 4x4 homogeneous
            poses.append(pose)
    return poses

poses = load_poses(pose_path)
T_current = poses[current_frame_id]
T_current_inv = np.linalg.inv(T_current)

# ─── 라벨 매핑 ─────────────────────────────
learning_map = {
    0: 0, 1: 0, 9: 1, 10: 1, 11: 1, 13: 1, 15: 1, 16: 1, 18: 1, 20: 1,
    30: 1, 31: 1, 32: 1, 40: 1, 44: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1,
    60: 1, 70: 1, 71: 1, 72: 1, 80: 1, 81: 1, 99: 1, 
    251: 2, 252: 2, 253: 2, 254: 2, 255: 2, 256: 2, 257: 2, 258: 2, 259: 2,
}

def relabel(pcds_labels, label_map):
    result_labels = np.zeros((pcds_labels.shape[0],), dtype=pcds_labels.dtype)
    for key in label_map:
        mask = pcds_labels == key
        result_labels[mask] = label_map[key]
    return result_labels

# ─── 2.5D Bounding Box 계산 (XY 평면 회전 가능, Z축 수직) ─────────────────────────────
def get_2d_obb(pcd):
    points = np.asarray(pcd.points)
    xy = points[:, :2]
    mean_xy = xy.mean(axis=0)
    xy_centered = xy - mean_xy

    # PCA (2x2)
    cov = np.cov(xy_centered.T)
    eigvals, eigvecs = np.linalg.eigh(cov)
    R_2d = eigvecs  # 2x2 회전 행렬
    rotated = xy_centered @ R_2d

    min_xy = rotated.min(axis=0)
    max_xy = rotated.max(axis=0)
    # AABB 코너 (회전 좌표계에서)
    corners = np.array([
        [min_xy[0], min_xy[1]],
        [max_xy[0], min_xy[1]],
        [max_xy[0], max_xy[1]],
        [min_xy[0], max_xy[1]],
    ])
    corners_orig = corners @ R_2d.T + mean_xy

    min_z = points[:, 2].min()
    max_z = points[:, 2].max()

    center_xy = corners_orig.mean(axis=0)
    center_z = (min_z + max_z) / 2.0
    center = np.array([center_xy[0], center_xy[1], center_z])

    extent_xy = max_xy - min_xy  # 길이, 너비 (회전 좌표계 상)
    extent_z = max_z - min_z      # 높이
    extent = np.array([extent_xy[0], extent_xy[1], extent_z])

    R_full = np.eye(3)
    R_full[:2, :2] = R_2d

    obb = o3d.geometry.OrientedBoundingBox(center, R_full, extent)
    return obb

# ─── DBSCAN 파라미터 ─────────────────────────────
dbscan_eps = 0.5
dbscan_min_points = 10

# ─── 컬러 매핑 (0: 초록, 1: 검정, 2: 빨강) ─────────────────────────────
color_map = {
    0: [0, 1, 0],
    1: [0, 0, 0],
    2: [1, 0, 0]
}

# ─── 누적 포인트 및 bounding box 저장용 리스트 ─────────────────────────────
accumulated_points_list = []  # 각 원소: (N x 3) 배열
accumulated_colors_list = []  # 각 원소: (N x 3) 배열
all_bboxes = []  # bounding box 리스트

# 현재 프레임의 전체 점군(색상 유지)을 저장하기 위한 변수
current_full_xyz = None
current_full_colors = None

# ─── 각 프레임 처리 (과거 8프레임 + 현재 프레임: 총 9프레임) ─────────────────────────────
for fid in range(current_frame_id - 20, current_frame_id + 1):
    file_id = f"{fid:06d}"
    bin_path = f"{velo_dir}/{file_id}.bin"
    label_path = f"{label_dir}/{file_id}.label"
    
    # 파일 읽기
    pcds_tmp = np.fromfile(bin_path, dtype=np.float32).reshape((-1, 4))
    pcds_label = np.fromfile(label_path, dtype=np.uint32).reshape((-1))
    sem_label = pcds_label & 0xFFFF
    label_vals = relabel(sem_label, learning_map)
    
    xyz = pcds_tmp[:, :3]
    dist = np.linalg.norm(xyz, axis=1)
    mask = (dist >= 2) & (dist <= 50)
    xyz = xyz[mask]
    label_vals = label_vals[mask]
    colors_frame = np.array([color_map[l] for l in label_vals])
    
    # 현재 프레임 전체 포인트(색상 유지) 저장
    if fid == current_frame_id:
        current_full_xyz = xyz.copy()
        current_full_colors = colors_frame.copy()
    
    # ── label 2인 점에 대해 DBSCAN 클러스터링 및 bounding box 계산 ──
    target_idx = (label_vals == 2)
    target_xyz = xyz[target_idx]
    if target_xyz.shape[0] > 0:
        target_pcd = o3d.geometry.PointCloud()
        target_pcd.points = o3d.utility.Vector3dVector(target_xyz)
        clusters = np.array(target_pcd.cluster_dbscan(eps=dbscan_eps, min_points=dbscan_min_points, print_progress=False))
        # 각 클러스터별 bounding box 계산
        for i in range(clusters.max() + 1):
            cluster_mask = clusters == i
            cluster_points = target_xyz[cluster_mask]
            if cluster_points.shape[0] == 0:
                continue
            cluster_pcd = o3d.geometry.PointCloud()
            cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)
            obb = get_2d_obb(cluster_pcd)
            # ── transform 전, 각 프레임 내의 bounding box는 프레임 로컬 좌표계 ──
            # ── 이후 현재 프레임 좌표계로 변환 ──
            # 색상: 현재 프레임은 빨강, 나머지는 파랑
            obb.color = (1, 0, 0) if fid == current_frame_id else (0, 0, 1)
            
            # ── bounding box의 rigid 변환 (커스텀 함수 사용) ──
            def transform_obb(obb, T):
                center_h = np.hstack((obb.center, 1))
                new_center = (T @ center_h)[:3]
                new_R = T[:3, :3] @ obb.R
                new_extent = obb.extent
                new_obb = o3d.geometry.OrientedBoundingBox(new_center, new_R, new_extent)
                new_obb.color = obb.color
                return new_obb
            
            T_frame = poses[fid]
            T_transform = T_current_inv @ T_frame
            obb_transformed = transform_obb(obb, T_transform)
            all_bboxes.append(obb_transformed)
    
    # ── label 1인 점만 누적 (샘플링) ──
    accum_idx = (label_vals == 1)
    if np.sum(accum_idx) > 0:
        sample_ratio = 0.05 
        num_samples = max(1, int(np.sum(accum_idx) * sample_ratio))
        sample_indices = np.random.choice(np.where(accum_idx)[0], num_samples, replace=False)
        sampled_xyz = xyz[sample_indices]
        sampled_colors = colors_frame[sample_indices]
        
        # 현재 프레임 좌표계로 변환: T_transform = T_current_inv @ T_frame
        T_frame = poses[fid]
        T_transform = T_current_inv @ T_frame
        ones = np.ones((sampled_xyz.shape[0], 1))
        sampled_xyz_h = np.hstack((sampled_xyz, ones))
        sampled_xyz_trans = (T_transform @ sampled_xyz_h.T).T[:, :3]
        
        accumulated_points_list.append(sampled_xyz_trans)
        accumulated_colors_list.append(sampled_colors)

# ─── 누적 포인트 클라우드 생성 (라벨별 색상 유지) ─────────────────────────────
if accumulated_points_list:
    all_points = np.vstack(accumulated_points_list)
    all_colors = np.vstack(accumulated_colors_list)
    accumulated_pcd = o3d.geometry.PointCloud()
    accumulated_pcd.points = o3d.utility.Vector3dVector(all_points)
    accumulated_pcd.colors = o3d.utility.Vector3dVector(all_colors)
else:
    accumulated_pcd = None

# ─── 현재 프레임 전체 포인트 클라우드 (필터링 및 원래 색상) ─────────────────────────────
if current_full_xyz is not None:
    current_pcd = o3d.geometry.PointCloud()
    current_pcd.points = o3d.utility.Vector3dVector(current_full_xyz)
    current_pcd.colors = o3d.utility.Vector3dVector(current_full_colors)
else:
    current_pcd = None

# ─── 최종 시각화: 누적 포인트, 현재 프레임 전체 포인트, bounding box ─────────────────────────────
geometries = []
if accumulated_pcd is not None:
    geometries.append(accumulated_pcd)
if current_pcd is not None:
    geometries.append(current_pcd)
for bbox in all_bboxes:
    geometries.append(bbox)

o3d.visualization.draw_geometries(geometries)


In [ ]:
import open3d as o3d
import numpy as np
import copy
import random

# ─── 파일 경로 및 기본 설정 ─────────────────────────────
seq = "08"  # 시퀀스 고정
data_dir = "/home/workspace/KITTI/dataset/sequences/" + seq
velo_dir = data_dir + "/velodyne"
label_dir = data_dir + "/labels"
pose_path = data_dir + "/poses.txt"

# 현재 프레임 id를 30으로 설정 (파일명은 6자리 zero padding)
current_frame_id = 2000

# ─── 포즈 파일 읽기 ─────────────────────────────
def load_poses(pose_file):
    poses = []
    with open(pose_file, 'r') as f:
        for line in f:
            vals = np.array(list(map(float, line.split())))
            pose = np.vstack((vals.reshape(3, 4), [0, 0, 0, 1]))  # 4x4 homogeneous
            poses.append(pose)
    return poses

poses = load_poses(pose_path)
T_current = poses[current_frame_id]
T_current_inv = np.linalg.inv(T_current)

# ─── 라벨 매핑 ─────────────────────────────
learning_map = {
    0: 0, 1: 0, 9: 1, 10: 1, 11: 1, 13: 1, 15: 1, 16: 1, 18: 1, 20: 1,
    30: 1, 31: 1, 32: 1, 40: 1, 44: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1,
    60: 1, 70: 1, 71: 1, 72: 1, 80: 1, 81: 1, 99: 1, 
    251: 2, 252: 2, 253: 2, 254: 2, 255: 2, 256: 2, 257: 2, 258: 2, 259: 2,
}

def relabel(pcds_labels, label_map):
    result_labels = np.zeros((pcds_labels.shape[0],), dtype=pcds_labels.dtype)
    for key in label_map:
        mask = pcds_labels == key
        result_labels[mask] = label_map[key]
    return result_labels

# ─── 2.5D Bounding Box 계산 (XY 평면 회전 가능, Z축 수직) ─────────────────────────────
def get_2d_obb(pcd):
    points = np.asarray(pcd.points)
    xy = points[:, :2]
    mean_xy = xy.mean(axis=0)
    xy_centered = xy - mean_xy

    # PCA (2x2)
    cov = np.cov(xy_centered.T)
    eigvals, eigvecs = np.linalg.eigh(cov)
    R_2d = eigvecs  # 2x2 회전 행렬
    rotated = xy_centered @ R_2d

    min_xy = rotated.min(axis=0)
    max_xy = rotated.max(axis=0)
    # 회전 좌표계에서 AABB의 코너
    corners = np.array([
        [min_xy[0], min_xy[1]],
        [max_xy[0], min_xy[1]],
        [max_xy[0], max_xy[1]],
        [min_xy[0], max_xy[1]],
    ])
    # 원래 좌표계로 복원
    corners_orig = corners @ R_2d.T + mean_xy

    min_z = points[:, 2].min()
    max_z = points[:, 2].max()

    center_xy = corners_orig.mean(axis=0)
    center_z = (min_z + max_z) / 2.0
    center = np.array([center_xy[0], center_xy[1], center_z])

    extent_xy = max_xy - min_xy  # 회전 좌표계 상 길이, 너비
    extent_z = max_z - min_z      # 높이
    extent = np.array([extent_xy[0], extent_xy[1], extent_z])

    R_full = np.eye(3)
    R_full[:2, :2] = R_2d

    obb = o3d.geometry.OrientedBoundingBox(center, R_full, extent)
    return obb

# ─── DBSCAN 파라미터 ─────────────────────────────
dbscan_eps = 0.5
dbscan_min_points = 10

# ─── 컬러 매핑 (0: 초록, 1: 검정, 2: 빨강) ─────────────────────────────
color_map = {
    0: [0, 1, 0],
    1: [0, 0, 0],
    2: [1, 0, 0]
}

# ─── 누적 포인트 및 bounding box 저장용 리스트 ─────────────────────────────
accumulated_points_list = []  # 각 원소: (N x 3) 배열
accumulated_colors_list = []  # 각 원소: (N x 3) 배열
all_bboxes = []  # bounding box 리스트

# 현재 프레임의 전체 점군(색상 유지)을 저장하기 위한 변수
current_full_xyz = None
current_full_colors = None

# ─── 각 프레임 처리 (과거 8프레임 + 현재 프레임: 총 9프레임) ─────────────────────────────
for fid in range(current_frame_id - 8, current_frame_id + 1):
    file_id = f"{fid:06d}"
    bin_path = f"{velo_dir}/{file_id}.bin"
    label_path = f"{label_dir}/{file_id}.label"
    
    # 파일 읽기
    pcds_tmp = np.fromfile(bin_path, dtype=np.float32).reshape((-1, 4))
    pcds_label = np.fromfile(label_path, dtype=np.uint32).reshape((-1))
    sem_label = pcds_label & 0xFFFF
    label_vals = relabel(sem_label, learning_map)
    
    xyz = pcds_tmp[:, :3]
    dist = np.linalg.norm(xyz, axis=1)
    mask = (dist >= 2) & (dist <= 50)
    xyz = xyz[mask]
    label_vals = label_vals[mask]
    colors_frame = np.array([color_map[l] for l in label_vals])
    
    # 현재 프레임 전체 포인트(색상 유지) 저장
    if fid == current_frame_id:
        current_full_xyz = xyz.copy()
        current_full_colors = colors_frame.copy()
    
    # ── 라벨 2인 점에 대해 DBSCAN 및 bounding box 계산 ──
    target_idx = (label_vals == 2)
    target_xyz = xyz[target_idx]
    if target_xyz.shape[0] > 0:
        target_pcd = o3d.geometry.PointCloud()
        target_pcd.points = o3d.utility.Vector3dVector(target_xyz)
        clusters = np.array(target_pcd.cluster_dbscan(eps=dbscan_eps, min_points=dbscan_min_points, print_progress=False))
        for i in range(clusters.max() + 1):
            cluster_mask = clusters == i
            cluster_points = target_xyz[cluster_mask]
            if cluster_points.shape[0] == 0:
                continue
            cluster_pcd = o3d.geometry.PointCloud()
            cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)
            obb = get_2d_obb(cluster_pcd)
            # ── transform 전: 각 프레임 내의 bounding box는 프레임 로컬 좌표계 ──
            obb.color = (1, 0, 0) if fid == current_frame_id else (0, 0, 1)
            
            # ── bounding box의 rigid 변환 (커스텀 함수) ──
            def transform_obb(obb, T):
                center_h = np.hstack((obb.center, 1))
                new_center = (T @ center_h)[:3]
                new_R = T[:3, :3] @ obb.R
                new_extent = obb.extent  # scaling 없음 가정
                new_obb = o3d.geometry.OrientedBoundingBox(new_center, new_R, new_extent)
                new_obb.color = obb.color
                return new_obb
            
            T_frame = poses[fid]
            T_transform = T_current_inv @ T_frame
            obb_transformed = transform_obb(obb, T_transform)
            all_bboxes.append(obb_transformed)
    
    # ── 누적 포인트: 라벨 1과 2 모두 (샘플링) ──
    accum_idx = ((label_vals == 1) | (label_vals == 2))
    if np.sum(accum_idx) > 0:
        sample_ratio = 0.1  # 10% 샘플링
        num_samples = max(1, int(np.sum(accum_idx) * sample_ratio))
        sample_indices = np.random.choice(np.where(accum_idx)[0], num_samples, replace=False)
        sampled_xyz = xyz[sample_indices]
        sampled_colors = colors_frame[sample_indices]
        
        T_frame = poses[fid]
        T_transform = T_current_inv @ T_frame
        ones = np.ones((sampled_xyz.shape[0], 1))
        sampled_xyz_h = np.hstack((sampled_xyz, ones))
        sampled_xyz_trans = (T_transform @ sampled_xyz_h.T).T[:, :3]
        
        accumulated_points_list.append(sampled_xyz_trans)
        accumulated_colors_list.append(sampled_colors)

# ─── 누적 포인트 클라우드 생성 (라벨별 색상 유지) ─────────────────────────────
if accumulated_points_list:
    all_points = np.vstack(accumulated_points_list)
    all_colors = np.vstack(accumulated_colors_list)
    accumulated_pcd = o3d.geometry.PointCloud()
    accumulated_pcd.points = o3d.utility.Vector3dVector(all_points)
    accumulated_pcd.colors = o3d.utility.Vector3dVector(all_colors)
else:
    accumulated_pcd = None

# ─── 현재 프레임 전체 포인트 클라우드 생성 (원래 색상) ─────────────────────────────
if current_full_xyz is not None:
    current_pcd = o3d.geometry.PointCloud()
    current_pcd.points = o3d.utility.Vector3dVector(current_full_xyz)
    current_pcd.colors = o3d.utility.Vector3dVector(current_full_colors)
else:
    current_pcd = None

# ─── 시각화 대상에서 라벨 1 (검정: [0,0,0]) 제거 ─────────────────────────────
def remove_label1(pcd):
    pts = np.asarray(pcd.points)
    cols = np.asarray(pcd.colors)
    # 라벨 1은 색상이 정확히 [0,0,0]으로 가정
    mask = ~np.all(cols == np.array([0,0,0]), axis=1)
    pcd_filtered = o3d.geometry.PointCloud()
    pcd_filtered.points = o3d.utility.Vector3dVector(pts[mask])
    pcd_filtered.colors = o3d.utility.Vector3dVector(cols[mask])
    return pcd_filtered

if accumulated_pcd is not None:
    accumulated_pcd = remove_label1(accumulated_pcd)
if current_pcd is not None:
    current_pcd = remove_label1(current_pcd)

# ─── 최종 시각화: 누적 포인트, 현재 프레임 전체 포인트, bounding box ─────────────────────────────
geometries = []
if accumulated_pcd is not None:
    geometries.append(accumulated_pcd)
if current_pcd is not None:
    geometries.append(current_pcd)
for bbox in all_bboxes:
    geometries.append(bbox)

o3d.visualization.draw_geometries(geometries)
